In [1]:
import io
import zipfile
import requests
import networkx as nx
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
url = ('https://github.com/ipython-books/'
       'cookbook-2nd-data/blob/master/'
       'road.zip?raw=true')
r = io.BytesIO(requests.get(url).content)
zipfile.ZipFile(r).extractall('data')
g = nx.read_shp('data/tl_2013_06_prisecroads.shp')

In [3]:
sgs = [g.subgraph(c) for c in nx.connected_components(g.to_undirected())]
i = np.argmax([len(sg) for sg in sgs])
sg = sgs[i]
len(sg)

464

In [4]:
def get_path(n0, n1):
    """If n0 and n1 are connected nodes in the graph,
    this function returns an array of point
    coordinates along the road linking these two
    nodes."""
    return np.array(json.loads(sg[n0][n1]['Json'])
                    ['coordinates'])


In [5]:
# from [https://stackoverflow.com/a/8859667/1595060](https://stackoverflow.com/a/8859667/1595060)
EARTH_R = 6372.8

def geocalc(lat0, lon0, lat1, lon1):
    """Return the distance (in km) between two points
    in geographical coordinates."""
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt((np.cos(lat1) * np.sin(dlon)) ** 2 +
        (np.cos(lat0) * np.sin(lat1) - np.sin(lat0) *
         np.cos(lat1) * np.cos(dlon)) ** 2)
    x = np.sin(lat0) * np.sin(lat1) + \
        np.cos(lat0) * np.cos(lat1) * np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c


In [6]:
def get_path_length(path):
    return np.sum(geocalc(path[1:, 1], path[1:, 0],
                          path[:-1, 1], path[:-1, 0]))

In [7]:
# Compute the length of the road segments.
for n0, n1 in sg.edges:
    path = get_path(n0, n1)
    distance = get_path_length(path)
    sg.edges[n0, n1]['distance'] = distance


In [8]:
#look for connected nodes
for ix, x in enumerate(list(sg.nodes)):
    for iy, y in enumerate(list(sg.nodes)):
        if nx.has_path(sg,x,y) and x != y:
            print(ix, iy)

0 19
0 37
0 59
0 82
0 151
0 183
0 239
0 258
0 262
0 297
0 352
0 364
0 368
0 388
0 404
0 409
0 411
0 431
0 447
0 448
0 451
0 452
1 443
2 77
3 144
3 454
4 448
5 50
5 137
5 140
5 161
5 231
5 268
5 339
5 363
5 375
6 63
6 113
6 135
6 146
6 158
6 163
6 169
6 217
6 224
6 228
6 230
6 243
6 271
6 288
6 302
6 304
6 305
6 315
6 337
6 346
6 350
6 354
6 359
6 362
6 416
8 122
8 217
8 224
8 337
8 416
9 65
9 73
9 89
9 126
9 181
9 185
9 189
9 199
9 200
9 210
9 245
9 318
9 320
9 335
9 341
9 343
9 345
9 391
9 393
9 407
9 439
9 453
10 187
10 249
11 141
11 163
11 240
12 25
12 209
12 227
13 137
13 140
14 193
14 324
15 19
15 168
15 174
15 195
15 366
15 371
15 378
15 425
15 459
17 28
17 41
17 71
17 81
17 95
17 97
17 158
17 166
17 197
17 206
17 229
17 235
17 261
17 271
17 282
17 305
17 334
17 348
17 370
17 401
17 433
17 463
18 119
18 209
18 222
18 275
18 402
18 403
20 137
20 140
20 231
20 399
21 79
21 136
21 232
21 248
21 253
21 386
21 445
24 247
25 209
26 23
26 49
26 64
26 131
26 205
26 347
26 356
26 365
26 3

256 24
256 40
256 51
256 176
256 190
256 216
256 247
256 254
256 313
256 418
256 426
258 297
258 352
258 451
259 159
260 7
260 54
260 88
260 443
261 41
262 37
262 151
262 409
262 431
263 135
263 230
263 354
264 347
264 454
265 22
265 148
265 337
265 416
266 37
266 76
266 151
266 262
266 409
266 431
267 251
267 303
268 137
268 140
268 231
268 339
269 46
269 47
269 77
269 93
269 115
269 283
269 338
269 353
269 380
269 398
270 322
270 394
273 301
275 119
275 222
275 402
275 403
276 64
278 93
278 255
279 42
279 105
279 112
279 177
279 219
280 43
280 84
280 93
280 109
280 115
280 160
280 173
280 176
280 180
280 249
280 251
280 267
280 303
280 380
280 408
280 456
280 457
281 159
281 377
282 158
282 271
282 305
283 77
284 14
284 172
284 193
284 324
284 462
285 42
285 105
285 112
285 177
285 219
285 279
285 338
286 238
286 322
286 349
286 407
287 46
287 83
287 91
287 93
287 99
287 109
287 110
287 115
287 127
287 160
287 173
287 186
287 198
287 207
287 249
287 251
287 267
287 272
287 289
287 30

In [13]:
#
pos0 = list(sg.nodes)[194]
pos1 = list(sg.nodes)[372]

In [14]:
nodes = np.array(sg.nodes())
# Get the closest nodes in the graph.
pos0_i = np.argmin(
    np.sum((nodes[:, ::-1] - pos0)**2, axis=1))
pos1_i = np.argmin(
    np.sum((nodes[:, ::-1] - pos1)**2, axis=1))


In [15]:
# Compute the shortest path.
path = nx.shortest_path(
    sg,
    source=tuple(pos0),
    target=tuple(pos1),
    weight='distance') #distance is from edge data shown below
len(path)


2

In [16]:
#take a look at the edges data
# we can see that we have a distance object in the json that we are using as the "weight" above
#we therefore need to replace this distance with a metric that includes the other information we are interested in
[e for e in sg.edges.data()][0]

((-121.586642, 39.14501299999999),
 (-121.59112900000001, 39.14465899999999),
 {'LINEARID': '1104493254357',
  'FULLNAME': 'State Rte 70',
  'RTTYP': 'S',
  'MTFCC': 'S1200',
  'ShpName': 'tl_2013_06_prisecroads',
  'Wkb': b"\x00\x00\x00\x00\x02\x00\x00\x00\x16\xc0^e\x8b\x8a\xe3\x1dq@C\x92\x8f\xc96?V\xc0^e\x8d\xdaH\xb6R@C\x92\x8f\x10\xa9\x9bn\xc0^e\x94\xd0\xdc\xfc\xc6@C\x92\x8c\xefg+\x87\xc0^e\x97$tS\x8f@C\x92\x8c?>\x03q\xc0^e\x99\xaa`\x91:@C\x92\x8b\xda\x945\xac\xc0^e\xa1@W\x08%@C\x92\x8a\xbd]\xc3\xff\xc0^e\xa3\xcau\x03\xb9@C\x92\x8aa\x17r\x0c\xc0^e\xa82\xb9\x90\xb0@C\x92\x8aX\xb3\xf6;\xc0^e\xaa\xc1\tJ,@C\x92\x8a\x04\xd1 \x18\xc0^e\xad\xe6W\xb8N@C\x92\x89\xa0'RS\xc0^e\xaf\xcc\xe1\xc5\x82@C\x92\x89m\xd2kq\xc0^e\xb2R\xce\x03/@C\x92\x88\xcep:\xfa\xc0^e\xb4\xbf\x8f\xcdh@C\x92\x88?\xd5\x02$\xc0^e\xb9\xfd\xbd/\xa2@C\x92\x87]V\xf3,\xc0^e\xbc\x8c\x0c\xe9\x1d@C\x92\x86\xf0I\xa9\x96\xc0^e\xbf\x05c\xed\x10@C\x92\x86\x9cf\xd3s\xc0^e\xc6y\xcct\xb9@C\x92\x85\xa0\xbeQ\x07\xc0^e\xc8\xf3#x\xab@C\x92\x

In [17]:
roads = pd.DataFrame(
    [sg.edges[path[i], path[i + 1]]
     for i in range(len(path) - 1)],
    columns=['FULLNAME', 'MTFCC',
             'RTTYP', 'distance'])
roads


,FULLNAME,MTFCC,RTTYP,distance
0,Golden State Fwy,S1100,M,22.014648


In [18]:
roads['distance'].sum()


22.014647832843167

In [19]:
def get_full_path(path):
    """Return the positions along a path."""
    p_list = []
    curp = None
    for i in range(len(path) - 1):
        p = get_path(path[i], path[i + 1])
        if curp is None:
            curp = p
        if (np.sum((p[0] - curp) ** 2) >
                np.sum((p[-1] - curp) ** 2)):
            p = p[::-1, :]
        p_list.append(p)
        curp = p[-1]
    return np.vstack(p_list)

In [20]:
#now we have a full gps path to plot
get_full_path(path)


array([[-118.627451,   34.502135],
       [-118.627433,   34.501895],
       [-118.627403,   34.501575],
       [-118.627386,   34.501488],
       [-118.627269,   34.500942],
       [-118.627178,   34.500666],
       [-118.627105,   34.50046 ],
       [-118.627041,   34.500315],
       [-118.626905,   34.499981],
       [-118.626767,   34.499693],
       [-118.626663,   34.499475],
       [-118.626549,   34.499267],
       [-118.626172,   34.498677],
       [-118.62586 ,   34.498272],
       [-118.625288,   34.497552],
       [-118.623987,   34.495782],
       [-118.623439,   34.495035],
       [-118.621699,   34.492738],
       [-118.621183,   34.492016],
       [-118.620909,   34.491657],
       [-118.619342,   34.489591],
       [-118.619052,   34.489214],
       [-118.618709,   34.488763],
       [-118.617309,   34.486884],
       [-118.617153,   34.486674],
       [-118.616686,   34.486022],
       [-118.616309,   34.485471],
       [-118.615884,   34.484752],
       [-118.615633,